In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pydicom
import imageio
import os
from glob import glob
import nibabel as nib
from tqdm import tqdm
import json
import nibabel as nib
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import nibabel as nib
from glob import glob
from tqdm import tqdm
import pandas as pd
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [3]:

data_dir='../../data/250513 C MR/'
save_dir='../../data/mri_nii/'
df=pd.read_csv('../../data/filter_mri_df.csv')


In [ ]:
folder_list=[]
for i in range(len(df)):
    folder_list.append(data_dir+df['ID'][i])
for i in tqdm(range(len(folder_list))):
    dicom_paths = glob(folder_list[i] + '/*.dcm')
    slices = []

    for dicom_file in sorted(dicom_paths):  # slice 순서 보존
        dcm = pydicom.dcmread(dicom_file)
        img = dcm.pixel_array.astype(np.float32)
    
        # RescaleSlope / Intercept 적용
        slope = float(getattr(dcm, 'RescaleSlope', 1))
        intercept = float(getattr(dcm, 'RescaleIntercept', 0))
        img = img * slope + intercept
        img=cv2.resize(img, (256, 256), interpolation=cv2.INTER_LINEAR)
        slices.append(img)

    volume = np.stack(slices, axis=0)  # (Z, H, W)
    patient_id = os.path.basename(folder_list[i]).replace('.dcm', '')

    # ✅ 정규화: -1 ~ 1
    vmin = np.percentile(volume, 5)
    vmax = np.percentile(volume, 100)

    volume = np.clip(volume, vmin, vmax)
    volume = (volume - vmin) / (vmax - vmin)  # → 0 ~ 1
    volume = volume * 2 - 1                   # → -1 ~ 1
    volume = volume.astype(np.float32)
    # NIfTI 저장
    nifti_img = nib.Nifti1Image(volume, affine=np.eye(4))
    save_path = os.path.join(save_dir, f"{patient_id}.nii.gz")
    create_dir(os.path.dirname(save_path))
    nib.save(nifti_img, save_path)

100%|██████████| 306/306 [03:03<00:00,  1.67it/s]
